<a href="https://colab.research.google.com/github/themysterysolver/QR-CIP/blob/main/CODE/PHASE_1_REFINED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy matplotlib opencv-python qrcode[pil] pycryptodome pillow

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import qrcode
import hashlib

- `cv2` is a part of *openCV* ***computer vision library***

In [3]:
def sha256_to_float(seed_string):
    hash_digest = hashlib.sha256(seed_string.encode()).hexdigest()
    print(hash_digest)
    hash_n = hash_digest[:16]
    return int(hash_n, 16) / 2**64

In [4]:
def lss_permutation(seed_string, n, r=3.9, s=3.0):#s is amplitutde,r-is logistic growth rate,pi-freq paramaeter
    x = sha256_to_float(seed_string)
    seq=[0]*n
    seq[0]=x
    for i in range(1,n):
      seq[i]=((r*seq[i-1]*(1-seq[i-1])) + s*np.sin(np.pi*seq[i-1]))%1
    permu_seq=np.argsort(seq)
    return np.array(permu_seq)

In [5]:
def crop_qr_border(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return img  # If no contours found, return the original image
    cnt = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(cnt)
    margin = 5
    x = max(0, x - margin)
    y = max(0, y - margin)
    w = min(img.shape[1] - x, w + 2 * margin)
    h = min(img.shape[0] - y, h + 2 * margin)
    return img[y:y+h, x:x+w]

In [6]:
def divide_qr(img):
    if img.shape[0] % 4 != 0 or img.shape[1] % 4 != 0:
        img = cv2.resize(img, (400, 400))
    block_size = img.shape[0] // 4
    return np.array([img[i * block_size:(i + 1) * block_size, j * block_size:(j + 1) * block_size] for i in range(4) for j in range(4)])

In [7]:
def scramble_qr(blocks, permutation):
    return blocks[permutation.tolist()] #converts into list

In [8]:
def rebuild_matrix(blocks):
    return np.vstack([np.hstack(blocks[i * 4:(i + 1) * 4]) for i in range(4)])

In [9]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

### AES
- (AES) **Advanced encryption Standard**
- AES works only on bytes
- `IV` is *iniialization vector*
- `CBC` is *cyber block chaining*
  - AES works with fixed-size blocks of `16 bytes` (128 bits).
  - if greater than that exist,it happens ***multiple rounds*** dividing into multiple *16byte block* .

<br>

- AES does XOR of plain text and previous $c_i$
- Encryption
$C_i = E_k ( P_i \oplus C_{i-1} )$
- when **no 1st** block
$C_1 = E_k ( P_1 \oplus IV )$
- Decrytption
$P_i = D_k ( C_i ) \oplus C_{i-1}$
- when no 1st block
$P_1 = D_k ( C_1 ) \oplus IV$

<br>

- AES I/P:
  - Plaintext (original data)
  - Secret key (16, 24, or 32 bytes)
  - Encryption process → Produces ciphertext (unreadable data)
  - Decryption process → Converts ciphertext back to plaintext using the same key.
<br>

## O/P
```
[ IV (16 bytes) ] + [ Encrypted Data (multiple of 16 bytes) ]

```

In [10]:
def encrypt_aes(data, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(data, AES.block_size))
    return cipher.iv + ct_bytes

In [14]:
def generate_shares(encrypted_data, n, k):
    shares = [np.random.randint(0, 256, size=len(encrypted_data), dtype=np.uint8) for _ in range(n-1)]
    last_share = np.frombuffer(encrypted_data, dtype=np.uint8).copy()
    for share in shares:
        last_share ^= share
    shares.append(last_share)
    return shares

In [18]:
def display_images(qr_image, shares, reconstructed_image, title_prefix=""):
    fig, axes = plt.subplots(1, len(shares), figsize=(15, 5))
    for i, share in enumerate(shares):
        share_size = int(np.ceil(np.sqrt(len(share))))
        padded_size = share_size * share_size
        if len(share) < padded_size:
            share_padded = np.pad(share, (0, padded_size - len(share)), mode='constant')
        else:
            share_padded = share[:padded_size]
        share_2d = share_padded.reshape((share_size, share_size))
        axes[i].imshow(share_2d, cmap='gray')
        axes[i].set_title(f'{title_prefix} Share {i + 1}')
        axes[i].axis('off')
    plt.show()


In [19]:
def reconstruct_image(shares):
    return np.bitwise_xor.reduce(shares).tobytes()

In [23]:
def descramble_qr(blocks, permutation):
    inverse_perm = np.argsort(permutation)
    return blocks[inverse_perm.tolist()]

In [24]:
def decrypt_aes(encrypted_data, key):
    iv = encrypted_data[:AES.block_size]
    ct = encrypted_data[AES.block_size:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ct), AES.block_size)

In [ ]:
if __name__ == "__main__":
    sha256_input = input("Enter a string for SHA-256-based permutation:")
    n=16
    permutation=lss_permutation(sha256_input,n)
    print(permutation)

    qr_input = input("Enter a string to generate a QR code: ")
    qr=qrcode.make(qr_input)
    qr.save("qr_code.png")

    img=cv2.imread("qr_code.png")
    print(img.shape)

    img = crop_qr_border(img)
    print(img.shape)

    img = cv2.resize(img, (400, 400))
    print(img.shape)

    divided_blocks = divide_qr(img)
    print(divided_blocks.shape)

    scrambled_blocks = scramble_qr(divided_blocks, permutation)
    print(scrambled_blocks.shape)

    scrambled_qr = rebuild_matrix(scrambled_blocks)
    print(scrambled_qr.shape)

    print("QR Code:")
    plt.imshow(img, cmap='gray')
    plt.title("QR Code")
    plt.axis('off')
    plt.show()

    print("Scrambled QR Code:")
    plt.imshow(scrambled_qr, cmap='gray')
    plt.title("Scrambled QR Code")
    plt.axis('off')
    plt.show()

    hash_digest = hashlib.sha256(qr_input.encode()).digest() #256-bits (32bytes)
    aes_key=hash_digest[:]
    print('aes_key:',hash_digest)
    encrypted_qr=encrypt_aes(scrambled_qr.tobytes(),hash_digest)#O/P:byes

    print("AES-Encrypted Scrambled QR Code:")
    encrypted_qr_array = np.frombuffer(encrypted_qr, dtype=np.uint8) #bytes->numpy array of uint8
    encrypted_qr_2d = encrypted_qr_array[:400*400].reshape((400, 400))  # Reshape to 2D for visualization
    plt.imshow(encrypted_qr_2d, cmap='gray')
    plt.title("AES-Encrypted Scrambled QR Code")
    plt.axis('off')
    plt.show()

    try:
        share_size = int(input("Input the number of shares images you want to create for encrypting (min is 2, max is 8): "))
        if share_size < 2 or share_size > 8:
            raise ValueError
    except ValueError:
        print("Input is not a valid integer!")
        exit(0)

    shares = generate_shares(encrypted_qr, share_size, share_size)
    print("Shares:")
    display_images(scrambled_qr, shares, scrambled_qr, "Scrambled")
    reconstructed_encrypted_qr = reconstruct_image(shares)

    print("AES-Encrypted Image After XORing Shares:")
    reconstructed_encrypted_qr_2d = np.frombuffer(reconstructed_encrypted_qr, dtype=np.uint8)[:400*400].reshape((400, 400))
    plt.imshow(reconstructed_encrypted_qr_2d, cmap='gray')
    plt.title("AES-Encrypted Image After XORing Shares")
    plt.axis('off')
    plt.show()

    sha256_input_verify = input("Enter the SHA-256-based password again to reconstruct the QR code: ")
    if sha256_input_verify == sha256_input:
        print("Password verified. Reconstructing the QR code...")

        decrypted_qr = decrypt_aes(reconstructed_encrypted_qr, aes_key)
        print(decrypted_qr)

        decrypted_qr_array = np.frombuffer(decrypted_qr, dtype=np.uint8).reshape(scrambled_qr.shape)
        print(decrypted_qr_array,decrypted_qr_array.shape)

        print("Scrambled QR Code from AES:")
        plt.imshow(decrypted_qr_array, cmap='gray')
        plt.title("Scrambled QR Code from AES")
        plt.axis('off')
        plt.show()

        descrambled_blocks = descramble_qr(divide_qr(decrypted_qr_array), permutation)
        descrambled_qr = rebuild_matrix(descrambled_blocks)

        print("Descrambled QR Code:")
        plt.imshow(descrambled_qr, cmap='gray')
        plt.title("Descrambled QR Code")
        plt.axis('off')
        plt.show()


- That's how a `290*290*3` array look like,290 rows and column with 3 cols each for ***rgb***
```
[
  [[255, 255, 255], [255, 255, 255], [255, 255, 255], ..., [255, 255, 255]],
  [[255, 255, 255], [0, 0, 0], [0, 0, 0], ..., [255, 255, 255]],
  [[255, 255, 255], [0, 0, 0], [0, 0, 0], ..., [255, 255, 255]],
  ...,
  [[255, 255, 255], [0, 0, 0], [0, 0, 0], ..., [255, 255, 255]]
]
```